In [38]:
import tkinter as tk
import time 
import numpy as np
import copy
from random import *
from matinv import *

# Q leaning:
# Environment: ALl possible grid positions, a total of 2^9 = 512


#Actions: clicking one of the 9 grids numbered 0 to 8
# Rewards:

#create a set of all states that are one move away from completing(total of 9 states):
# Each action can be represented as a tuple (i,j) which indicated i-th row, j-th column grid is the button to be clicked.
def states_with_actions(n):
    states_dict = {}
    for i in range(2**n):
        binary_array = []
        for j in range(n):
            binary_array.append((i >> j) & 1)
        for x in range(5):
            for y in range(5):
                states_dict[(tuple([tuple(binary_array[i:i+5]) for i in range(0,25,5)]), (x,y))] = 0
    return states_dict

#Reward function:
#--------------------
#After every move, this function compares the current grid state and the initial grid state to compare how many 
#lights they have in common using bitwise XOR. The negative of sum of this array will be the reward for that state. 
#Note that when the initial grid and the current grid state are the same, the reward is zero (Maximum reward). 
#-----------------------

def special_states():
    arrays = []
    for row in range(3):
        for col in range(3):
            empty_grid = [[0,0,0] for _ in range(3)]
            possible_moves = [
                (row - 2, col - 1),
                (row - 2, col + 1),
                (row - 1, col - 2),
                (row - 1, col + 2),
                (row + 1, col - 2),
                (row + 1, col + 2),
                (row + 2, col - 1),
                (row + 2, col + 1)
            ]
            empty_grid[row][col] ^= 1
            for move_row, move_col in possible_moves:
                if 0 <= move_row < 5 and 0 <= move_col < 5:
                    empty_grid [move_row][move_col] ^= 1
            arrays.append(empty_grid)
    return arrays

spl_grids = special_states()

def reward_for_state(list_of_lists):
    total = sum([sum(item) for item in list_of_lists])
    if total == 0:
        return 100
    elif list_of_lists in spl_grids:
        return -1
    elif (total > 3):
        return (-1)*total
    else:
        return -5


# Helper functions for the model

#Function to find whether a given state is terminal
def is_terminal(list_of_lists):
    if reward_for_state(list_of_lists) == 100:
        return True
    else:
        return False

    
#Function to find the max q-value of a state over all actions

def best_action(tuple1, my_dict):
    max_value = float('-inf')  # Initialize with negative infinity
    max_key = None

    for key, value in my_dict.items():
        if key[0] == tuple1 and value > max_value:
            max_value = value
            max_key = key[1]

    return max_key, max_value

def start_action():
    return (np.random.randint(3),np.random.randint(3))

#Epsilon greedy algorithm for choosing the next action
def next_action(grid, epsilon):
    if np.random.random() < epsilon:
        key, value = best_action(grid, q_values)
        return key
    else:
        return (np.random.randint(3),np.random.randint(3))

# Function imitating the action of clicking a grid to toggle the states of itself and its neighbors
def action_click(action):
    row = action[0]
    col = action[1]
    grid_value[row][col] ^= 1
    if row - 1 >= 0:
        grid_value[row - 1][col] ^= 1  # Cell above
    if row + 1 <= 2:
        grid_value[row + 1][col] ^= 1  # Cell below
    if col - 1 >= 0:
        grid_value[row][col - 1] ^= 1  # Cell to the left
    if col + 1 <= 2:
        grid_value[row][col + 1] ^= 1  # Cell to the right

    
#Function to get the solution



# Training
q_values = states_with_actions(9)

epsilon = 0.9
discount = 0.9
learning_rate = 0.9


start = time.time()               
for episode in range(10000):
    initial_grid = []
    for _ in range(3):
        initial_grid.append([randint(0,1) for b in range(0,3)])
    grid_value = copy.deepcopy(initial_grid)
    #print(grid_value)
    state = start_action()
    action_click(state)
    while not is_terminal(grid_value):
        #print(grid_value)
        temp_grid = tuple([tuple(i) for i in grid_value])
        next_act = next_action(temp_grid, epsilon) #next action to take based off of epsilon greedy algorithm
        action_click(next_act)
        reward = reward_for_state(grid_value)
        old_q_value = q_values[(temp_grid, next_act)]
        tuple1 = tuple([tuple(i) for i in grid_value])
        key, max_q_value = best_action(tuple1, q_values) 
        temporal_diff = reward + (discount * max_q_value) - old_q_value
        new_q_value = old_q_value + (learning_rate * temporal_diff)
        q_values[(temp_grid, next_act)] = new_q_value
    
print('Done')
end = time.time()
print(end-start)
# def on_grid_click(row, col):
#     grid_value[row][col] ^= 1
#     if row - 1 >= 0:
#         grid_value[row - 1][col] ^= 1  # Cell above
#     if row + 1 <= 2:
#         grid_value[row + 1][col] ^= 1  # Cell below
#     if col - 1 >= 0:
#         grid_value[row][col - 1] ^= 1  # Cell to the left
#     if col + 1 <= 2:
#         grid_value[row][col + 1] ^= 1  # Cell to the right 
#     update_grid()
    
#     max_moves_counter.config(text=f"Moves Needed: {sum(solver_3(grid_value))}")
    
# def on_enter(event, row, col):
#     canvas.itemconfig(grid_cells[row][col], fill='yellow')

# def on_leave(event, row, col):
#     cell_value = grid_value[row][col]
#     if cell_value == 1:
#         cell_color = 'blue'
#     else:
#         cell_color = 'white'
#     canvas.itemconfig(grid_cells[row][col], fill=cell_color)
    
# def update_grid():
#     for row in range(3):
#         for col in range(3):
#             cell_value = grid_value[row][col]
#             if cell_value == 1:
#                 cell_color = 'blue'
#             else:
#                 cell_color = 'white'
#             canvas.itemconfig(grid_cells[row][col], fill=cell_color)

# def clear_grid():
#     global grid_value
#     grid_value = copy.deepcopy(initial_grid)
#     max_moves_counter.config(text=f"Moves Left: {sum(solver_3(grid_value))}")
#     update_grid()
#     root.update()
    
    
# def highlight_cell(row, col):
#     canvas.itemconfig(grid_cells[row][col], fill='yellow')
#     root.update()
#     time.sleep(0.5)  
#     canvas.itemconfig(grid_cells[row][col], fill='white')
#     root.update()
    
    
# def bot_instruction():
#     solution = solver_3(grid_value)
#     sol_mat = [solution[i:i+3] for i in range(0, len(solution), 3)]
#     for row in range(3):
#         for col in range(3):
#             if sol_mat[row][col] == 1:
#                 highlight_cell(row, col)
#                 on_grid_click(row,col)
#                 root.update()
#                 time.sleep(1)
            
    
# def new_game():
#     global initial_grid, grid_value
#     initial_grid = []
#     for _ in range(3):
#         initial_grid.append([randint(0,1) for b in range(0,3)])
#     grid_value = copy.deepcopy(initial_grid)
#     update_grid()
#     root.update()
    
    
# # Create the main window
# root = tk.Tk()
# root.title("Lights Out- 3 by 3")

# # Create a canvas to draw the grid
# canvas = tk.Canvas(root, width=300, height=300)
# canvas.pack()

# # Initialize grid value
# # 1 - Blue, 0 - White

# # Grid Creation
# cell_size = 100
# grid_cells = []
# for row in range(3):
#     grid_cells.append([])
#     for col in range(3):
#         x0, y0 = col * cell_size, row * cell_size
#         x1, y1 = x0 + cell_size, y0 + cell_size
#         cell = canvas.create_rectangle(x0, y0, x1, y1, fill= 'white', outline="black")
#         grid_cells[row].append(cell)
#         canvas.tag_bind(cell, "<Button-1>", lambda event, row=row, col=col: on_grid_click(row, col))
#         canvas.tag_bind(cell, "<Enter>", lambda event, row=row, col=col: on_enter(event, row, col))
#         canvas.tag_bind(cell, "<Leave>", lambda event, row=row, col=col: on_leave(event, row, col))
        
# clear_button = tk.Button(root, text="Restart", command=clear_grid)
# clear_button.pack()

# bot_button = tk.Button(root, text="Auto solver", command=bot_instruction)
# bot_button.pack()

# puzzle_button = tk.Button(root, text="New pattern", command=new_game)
# puzzle_button.pack()




# #print_button = tk.Button(root, text="Print Grid State", command=print_grid_state)
# #print_button.pack()

# # Start the Tkinter event loop
# #new_game()
# initial_grid = [[1,1,0],[0,1,1],[1,0,0]]
# #for _ in range(3):
#         #initial_grid.append([randint(0,1) for b in range(0,3)])
# grid_value = copy.deepcopy(initial_grid)
# update_grid()
# max_moves_counter = tk.Label(root, text=f"Moves Needed: {sum(solver_3(grid_value))}")
# max_moves_counter.pack()

# root.mainloop()

Done
50.173983573913574


In [41]:
initial_grid = [[0,0,1],[1,1,1],[1,0,0]]
grid_value = copy.deepcopy(initial_grid)

def solution(grid_value):
    if is_terminal(grid_value):
        return []
    else:
        state = (np.random.randint(3),np.random.randint(3))
        solution = []
        solution.append(state)
        action_click(state)
        while not is_terminal(grid_value):
            grid = tuple([tuple(i) for i in grid_value])
            next_act, val = best_action(grid, q_values)
            action_click(next_act)
            solution.append(next_act)
        return solution

solution(grid_value)

[(1, 2), (1, 0), (2, 2), (1, 1), (0, 0)]

In [1]:
def states_with_actions(n):
    states_dict = {}
    for i in range(2**n):
        binary_array = []
        for j in range(n):
            binary_array.append((i >> j) & 1)
        for x in range(5):
            for y in range(5):
                states_dict[(tuple([tuple(binary_array[i:i+5]) for i in range(0,25,5)]), (x,y))] = 0
    return states_dict

q_values = states_with_actions(25)

KeyboardInterrupt: 

In [2]:
q_values

NameError: name 'q_values' is not defined

In [ ]:
import pickle

with open('all_states_KO.pkl', 'wb') as f:
    pickle.dump(q_values, f)
        
with open('all_states_KO.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)